In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from stop_words import get_stop_words
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import gensim
import glob2
from itertools import chain
import os
import numpy as np
import itertools
import cPickle as pickle
from  scipy.stats import rankdata
import copy
from openpyxl import Workbook
from openpyxl.compat import range
from openpyxl.utils import get_column_letter
from openpyxl import load_workbook
import pandas as pd
from scipy.stats import hmean
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize


2018-03-21 14:48:57,279 : INFO : 'pattern' package not found; tag filters are not available for English


In [3]:
# ignores everything except english alphabet and  
def only_alphabet(text):
    return ''.join(i for i in text if (ord(i)<123 and ord(i)>96) or (ord(i)<91 and ord(i)>64) or ord(i)==32) 

In [4]:
full_filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        full_filenames.append(filename)
            
filenames = []
x= ['LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt','LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt','LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt' ,'LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt', 'LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt']
for glob_filenames in x:
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        filenames.append(os.path.basename(filename))

en_stop = set(get_stop_words('en'))
ps = PorterStemmer()

# "yield" for each file return token list  i.e list of lists
def files_to_tokens(glob_filenames):
    glob_filenames = os.path.normpath(glob_filenames)
    for filename in sorted(glob2.glob(glob_filenames)):
        f = open(filename)
        # read the whole file as lowercase string
        string = only_alphabet(f.read()).lower()
        tokens = []
        # tokenize that string
        for word in word_tokenize(string):
            if word not in en_stop:
                tokens.append(ps.stem(word))

        yield tokens
        f.close()


        
# yields token list for files specific to courts; needed for creating dictionaries
class texts:
    def DCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/**/*.txt')
    def NCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt')
    def SCDRC(self):
        return files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt')
    def DelhiHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt')
    def JharkhandHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt')
    def JodhpurHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt')
    def KolkataHC(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt')
    def SupremeCourt(self):
        return files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt')

# yields bow for each file - tuples id,fq ; needed to train models   
class my_corpus:    
    def DCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/DCDRC/*.txt'):
            yield dictionary.doc2bow(text)
    def NCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/NCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)            
    def SCDRC(self):
        for text in files_to_tokens('LegalAdhocTask/ConsumerCourtAdhocdata/SCDRC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def DelhiHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/DelhiHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JharkhandHC(self):
        for text in files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JharkhandHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def JodhpurHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/JodhpurHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def KolkataHC(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/KolkataHC/**/*.txt'):
            yield dictionary.doc2bow(text)
    def SupremeCourt(self):
        for text in  files_to_tokens('LegalAdhocTask/SC-HCAdhocData/SupremeCourt/**/*.txt'):
            yield dictionary.doc2bow(text)
    def everything(self):
        return chain(self.DCDRC(), self.NCDRC(), self.SCDRC(), self.DelhiHC(),
                     self.JharkhandHC(), self.JodhpurHC(), self.KolkataHC(), self.SupremeCourt())
        
        

In [5]:
#################################### laod the dictinary
dictionary = corpora.Dictionary.load('stem100_dictionary')

2018-03-21 14:49:28,680 : INFO : loading Dictionary object from stem100_dictionary
2018-03-21 14:49:28,921 : INFO : loaded stem100_dictionary


In [ ]:
######################################## load BOW
corpus = corpora.MmCorpus('stem100.mm')

2018-03-21 14:49:29,723 : INFO : loaded corpus index from stem100.mm.index
2018-03-21 14:49:29,724 : INFO : initializing corpus reader from stem100.mm
2018-03-21 14:49:29,769 : INFO : accepted corpus with 351985 documents, 40069 features, 92572684 non-zero entries


In [ ]:
%%time
from gensim.models import ldamodel
lda_model = ldamodel.LdaModel(corpus, num_topics=200, alpha = 'auto', id2word = dictionary, passes =3,  minimum_probability =0.0)

2018-03-21 14:49:29,777 : INFO : using autotuned alpha, starting with [0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000000000000001, 0.0050000

2018-03-21 14:51:29,477 : INFO : merging changes from 2000 documents into a model of 351985 documents
2018-03-21 14:51:35,069 : INFO : topic #154 (0.005): 0.017*"complaint" + 0.016*"joint" + 0.014*"memo" + 0.012*"complain" + 0.011*"polici" + 0.011*"parti" + 0.010*"dispos" + 0.009*"term" + 0.009*"consum" + 0.009*"henc"
2018-03-21 14:51:35,070 : INFO : topic #109 (0.005): 0.022*"op" + 0.017*"complain" + 0.014*"oppon" + 0.014*"siddappa" + 0.013*"polici" + 0.013*"deceas" + 0.011*"dr" + 0.010*"insur" + 0.009*"fact" + 0.009*"part"
2018-03-21 14:51:35,071 : INFO : topic #51 (0.008): 0.062*"complain" + 0.026*"opposit" + 0.025*"parti" + 0.017*"amount" + 0.016*"complaint" + 0.015*"rs" + 0.012*"op" + 0.010*"file" + 0.010*"date" + 0.009*"order"
2018-03-21 14:51:35,072 : INFO : topic #26 (0.008): 0.048*"complain" + 0.033*"oppon" + 0.019*"date" + 0.018*"complaint" + 0.018*"amount" + 0.015*"op" + 0.014*"rs" + 0.012*"said" + 0.010*"file" + 0.010*"interest"
2018-03-21 14:51:35,073 : INFO : topic #5 (0.

2018-03-21 14:52:45,933 : INFO : merging changes from 2000 documents into a model of 351985 documents
2018-03-21 14:52:51,020 : INFO : topic #145 (0.005): 0.029*"tax" + 0.018*"complain" + 0.015*"produc" + 0.014*"dharwad" + 0.012*"ele" + 0.012*"depart" + 0.011*"order" + 0.010*"crop" + 0.010*"op" + 0.009*"st"
2018-03-21 14:52:51,021 : INFO : topic #184 (0.005): 0.022*"produc" + 0.019*"complain" + 0.016*"dharwad" + 0.016*"op" + 0.015*"joshi" + 0.011*"decad" + 0.010*"compens" + 0.009*"kvkrishna" + 0.008*"forum" + 0.008*"ceil"
2018-03-21 14:52:51,022 : INFO : topic #162 (0.009): 0.107*"respond" + 0.064*"complain" + 0.028*"rs" + 0.014*"exp" + 0.011*"amount" + 0.010*"document" + 0.010*"complaint" + 0.010*"point" + 0.009*"servic" + 0.008*"r"
2018-03-21 14:52:51,023 : INFO : topic #115 (0.009): 0.038*"complain" + 0.016*"op" + 0.014*"complaint" + 0.012*"rs" + 0.012*"respond" + 0.011*"date" + 0.010*"amount" + 0.009*"file" + 0.009*"forum" + 0.008*"order"
2018-03-21 14:52:51,024 : INFO : topic #154

2018-03-21 14:54:03,093 : INFO : merging changes from 2000 documents into a model of 351985 documents
2018-03-21 14:54:07,869 : INFO : topic #94 (0.005): 0.030*"csi" + 0.029*"complain" + 0.021*"collaps" + 0.014*"derelict" + 0.014*"bulg" + 0.013*"op" + 0.012*"parti" + 0.012*"produc" + 0.011*"henc" + 0.011*"slide"
2018-03-21 14:54:07,870 : INFO : topic #14 (0.005): 0.029*"complain" + 0.023*"produc" + 0.018*"shekhargouda" + 0.018*"dharwad" + 0.015*"extent" + 0.015*"smtmanjula" + 0.014*"joinder" + 0.012*"declar" + 0.011*"import" + 0.011*"non"
2018-03-21 14:54:07,871 : INFO : topic #115 (0.011): 0.036*"complain" + 0.021*"udupi" + 0.014*"complaint" + 0.010*"parti" + 0.010*"date" + 0.009*"file" + 0.009*"rs" + 0.009*"point" + 0.008*"forum" + 0.008*"thecomplain"
2018-03-21 14:54:07,875 : INFO : topic #109 (0.016): 0.071*"hanag" + 0.023*"op" + 0.016*"exr" + 0.016*"sri" + 0.014*"complain" + 0.013*"occ" + 0.011*"naidu" + 0.010*"deceas" + 0.010*"insur" + 0.010*"bellari"
2018-03-21 14:54:07,876 : IN

2018-03-21 14:55:17,432 : INFO : merging changes from 2000 documents into a model of 351985 documents
2018-03-21 14:55:22,002 : INFO : topic #176 (0.005): 0.039*"english" + 0.038*"languag" + 0.027*"mistak" + 0.026*"complain" + 0.015*"yallappa" + 0.014*"yield" + 0.014*"insur" + 0.012*"op" + 0.012*"complaint" + 0.011*"actual"
2018-03-21 14:55:22,003 : INFO : topic #145 (0.005): 0.082*"tax" + 0.043*"chanc" + 0.024*"santosh" + 0.021*"incom" + 0.020*"dthaveri" + 0.016*"villag" + 0.015*"complain" + 0.014*"bangalor" + 0.014*"order" + 0.014*"gangavva"
2018-03-21 14:55:22,004 : INFO : topic #115 (0.014): 0.033*"complain" + 0.014*"complaint" + 0.014*"thecomplain" + 0.011*"parti" + 0.010*"order" + 0.010*"date" + 0.009*"rs" + 0.009*"file" + 0.008*"llb" + 0.008*"point"
2018-03-21 14:55:22,005 : INFO : topic #109 (0.022): 0.047*"hanag" + 0.022*"op" + 0.015*"sri" + 0.013*"complain" + 0.011*"exr" + 0.010*"naidu" + 0.008*"occ" + 0.008*"deceas" + 0.008*"insur" + 0.008*"investig"
2018-03-21 14:55:22,006 

...

2018-03-21 17:08:33,133 : INFO : merging changes from 2000 documents into a model of 351985 documents
2018-03-21 17:08:35,168 : INFO : topic #28 (0.007): 0.131*"amareshwari" + 0.085*"advccomplain" + 0.072*"dthaveri" + 0.061*"hudsan" + 0.041*"tqhangal" + 0.040*"puttappa" + 0.039*"hadsan" + 0.038*"dthaveribi" + 0.036*"gejjih" + 0.032*"advoppon"
2018-03-21 17:08:35,169 : INFO : topic #34 (0.009): 0.168*"orderdict" + 0.138*"ruler" + 0.137*"sbcwp" + 0.085*"lot" + 0.075*"travancor" + 0.044*"civilian" + 0.026*"abadi" + 0.021*"chowkidar" + 0.019*"opponentbank" + 0.014*"ips"
2018-03-21 17:08:35,170 : INFO : topic #137 (0.384): 0.085*"appeal" + 0.030*"judgment" + 0.020*"held" + 0.018*"order" + 0.017*"decis" + 0.016*"commiss" + 0.016*"court" + 0.015*"fact" + 0.014*"view" + 0.014*"therefor"
2018-03-21 17:08:35,171 : INFO : topic #154 (0.459): 0.011*"nrupatunga" + 0.007*"bellari" + 0.005*"docdocnoconsumercourtdcdrcdocnotextbefor" + 0.005*"dakshina" + 0.005*"governor" + 0.004*"tumkur" + 0.004*"advoc

2018-03-21 17:09:37,876 : INFO : merging changes from 2000 documents into a model of 351985 documents
2018-03-21 17:09:40,056 : INFO : topic #28 (0.007): 0.131*"amareshwari" + 0.084*"advccomplain" + 0.072*"dthaveri" + 0.060*"hudsan" + 0.041*"tqhangal" + 0.040*"puttappa" + 0.039*"hadsan" + 0.038*"dthaveribi" + 0.036*"gejjih" + 0.032*"advoppon"
2018-03-21 17:09:40,057 : INFO : topic #34 (0.009): 0.163*"orderdict" + 0.135*"ruler" + 0.134*"sbcwp" + 0.092*"lot" + 0.080*"travancor" + 0.045*"civilian" + 0.026*"abadi" + 0.020*"chowkidar" + 0.018*"opponentbank" + 0.014*"ips"
2018-03-21 17:09:40,059 : INFO : topic #20 (0.359): 0.088*"forum" + 0.080*"consum" + 0.059*"order" + 0.047*"district" + 0.045*"presid" + 0.042*"member" + 0.036*"file" + 0.033*"disput" + 0.031*"redress" + 0.024*"dismiss"
2018-03-21 17:09:40,060 : INFO : topic #154 (0.495): 0.009*"nrupatunga" + 0.005*"bellari" + 0.005*"advoc" + 0.005*"docdocnoconsumercourtdcdrcdocnotextbefor" + 0.004*"dakshina" + 0.004*"pointno" + 0.004*"gove

2018-03-21 17:10:57,321 : INFO : merging changes from 2000 documents into a model of 351985 documents
2018-03-21 17:10:59,499 : INFO : topic #28 (0.007): 0.327*"bhavnagar" + 0.079*"amareshwari" + 0.051*"advccomplain" + 0.044*"dthaveri" + 0.037*"hudsan" + 0.027*"sahkari" + 0.025*"tqhangal" + 0.024*"puttappa" + 0.024*"hadsan" + 0.023*"dthaveribi"
2018-03-21 17:10:59,500 : INFO : topic #34 (0.009): 0.177*"opponentin" + 0.108*"orderdict" + 0.090*"ruler" + 0.089*"sbcwp" + 0.077*"lot" + 0.057*"tooppon" + 0.053*"travancor" + 0.036*"opponentbank" + 0.030*"civilian" + 0.022*"atexh"
2018-03-21 17:10:59,501 : INFO : topic #20 (0.408): 0.093*"forum" + 0.082*"consum" + 0.059*"order" + 0.046*"district" + 0.045*"presid" + 0.043*"member" + 0.037*"file" + 0.035*"disput" + 0.032*"redress" + 0.023*"dismiss"
2018-03-21 17:10:59,502 : INFO : topic #154 (0.540): 0.016*"saurashtra" + 0.008*"nanal" + 0.007*"nrupatunga" + 0.006*"advoc" + 0.004*"docdocnoconsumercourtdcdrcdocnotextbefor" + 0.004*"bellari" + 0.00

2018-03-21 17:13:28,395 : INFO : merging changes from 2000 documents into a model of 351985 documents
2018-03-21 17:13:30,594 : INFO : topic #28 (0.007): 0.326*"bhavnagar" + 0.079*"amareshwari" + 0.051*"advccomplain" + 0.044*"dthaveri" + 0.037*"hudsan" + 0.027*"sahkari" + 0.025*"tqhangal" + 0.024*"puttappa" + 0.023*"hadsan" + 0.023*"dthaveribi"
2018-03-21 17:13:30,596 : INFO : topic #34 (0.009): 0.170*"opponentin" + 0.103*"orderdict" + 0.086*"ruler" + 0.085*"sbcwp" + 0.078*"lot" + 0.056*"tooppon" + 0.051*"travancor" + 0.042*"opponentbank" + 0.030*"civilian" + 0.021*"atexh"
2018-03-21 17:13:30,597 : INFO : topic #20 (0.465): 0.094*"forum" + 0.083*"consum" + 0.058*"order" + 0.046*"district" + 0.045*"presid" + 0.043*"member" + 0.037*"file" + 0.036*"disput" + 0.033*"redress" + 0.023*"dismiss"
2018-03-21 17:13:30,598 : INFO : topic #35 (0.537): 0.043*"act" + 0.029*"section" + 0.016*"provis" + 0.016*"court" + 0.015*"law" + 0.014*"may" + 0.013*"can" + 0.010*"power" + 0.010*"shall" + 0.010*"ru

2018-03-21 17:14:37,109 : INFO : merging changes from 2000 documents into a model of 351985 documents
2018-03-21 17:14:39,301 : INFO : topic #28 (0.007): 0.326*"bhavnagar" + 0.079*"amareshwari" + 0.051*"advccomplain" + 0.044*"dthaveri" + 0.037*"hudsan" + 0.027*"sahkari" + 0.025*"tqhangal" + 0.024*"puttappa" + 0.023*"hadsan" + 0.023*"dthaveribi"
2018-03-21 17:14:39,303 : INFO : topic #34 (0.009): 0.164*"opponentin" + 0.100*"orderdict" + 0.087*"lot" + 0.084*"ruler" + 0.082*"sbcwp" + 0.054*"tooppon" + 0.049*"travancor" + 0.041*"opponentbank" + 0.029*"civilian" + 0.021*"atexh"
2018-03-21 17:14:39,304 : INFO : topic #35 (0.519): 0.044*"act" + 0.029*"section" + 0.017*"provis" + 0.016*"court" + 0.015*"law" + 0.014*"may" + 0.013*"can" + 0.010*"power" + 0.010*"rule" + 0.010*"shall"
2018-03-21 17:14:39,305 : INFO : topic #20 (0.526): 0.095*"forum" + 0.082*"consum" + 0.058*"order" + 0.047*"presid" + 0.044*"member" + 0.043*"district" + 0.038*"disput" + 0.037*"file" + 0.034*"redress" + 0.022*"dismi

In [ ]:
# for each query : for every document get topic distribution

In [ ]:
lda_model.save('lda_200.model')

In [ ]:
lda_model = gensim.models.LdaModel.load('lda_200.model')

In [ ]:
lda_model.show_topics(-1)

In [ ]:
%%time
theta, _ = lda_model.inference(corpus)

In [ ]:
%%time
theta /= theta.sum(axis=1)[:, None]

In [ ]:
#Save
with open('thetalda_200.pkl', 'wb') as output:
    pickle.dump(theta, output, pickle.HIGHEST_PROTOCOL)

In [ ]:
# load 
with open('thetalda_200.pkl', 'rb') as ip:
    theta = pickle.load(ip)



In [ ]:
%%time
topics_terms = lda_model.state.get_lambda()
topics_terms = np.apply_along_axis(lambda x: x/x.sum(),1,topics_terms)


In [ ]:
lda_model[]

In [ ]:
lda_model.get_term_topics('water')

In [ ]:
# get the query truth in dictionary
query_truth = {}
for i in range(1,11):
    query_truth[str(i)]=[]
    
    
f = open('LegalAdhocTask/Consumer.qrels')
lines = [line.rstrip('\n').split("\t") for line in f]
for line in lines:
    del line[1]
    query_truth[line[0]].append(line[1:])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gensim.matutils as gm

In [ ]:
def sim_matrix(query, sim_fn):
    docs = query_truth[str(query)]
    docs.sort(key = lambda x : -int(x[1]))
    n = 0
    for i in docs:
        n = n + int(i[1])
    s  = len(docs)
    m = np.zeros((s,s))
    for i in xrange(s):
        for j in xrange(i + 1, s):
            v1 = theta[filenames.index(docs[i][0] + '.txt')]
            v2 = theta[filenames.index(docs[j][0] + '.txt')]
            if sim_fn == gm.cossim:
                v1 = gm.dense2vec(v1)
                v2 = gm.dense2vec(v2)
            m[i, j] = (1 - sim_fn(v1, v2))
    #print m
    print 'Query', query
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 10) 
    ax.matshow(m, cmap=plt.cm.Blues)
    ax.hlines(n -0.5, n -0.5, s - 0.5, lw=2, color='r')
    ax.vlines(n -0.5, -0.5, n - 0.5, lw=2, color='r')
    plt.show()


In [ ]:
# %%time
# for query in xrange(1, 11):
#     sim_matrix(query, gm.cossim)

In [ ]:
# %%time
# for query in xrange(1, 11):
#     sim_matrix(query, gm.jensen_shannon)

In [ ]:
%%time
dl = []
for i in dictionary.iteritems():
    dl.append((i[1], dictionary.dfs[i[0]]))
    
dl.sort(key = lambda x : -x[1])    

In [ ]:
dl

In [ ]:
sorted(dictionary.items(), key=lambda x: -x[1])

In [ ]:
%%time
for query in xrange(1, 11):
    sim_matrix(query, gm.hellinger)

In [ ]:
queries = [0]
for i in files_to_tokens('LegalAdhocTask/q*.txt'):
    queries.append(i)

In [ ]:
for i in queries[1:]:
    x  = lda_model[dictionary.doc2bow(i)]
    x.sort(key = lambda x: -x[1])
    print lda_model.print_topic(x[0][0]) + '\n'

In [ ]:

import math
from six import iteritems
from six.moves import xrange



class Lda(object):

    def __init__(self, corpus):
        self.corpus_size = dictionary.num_docs
        s = 0
        for i in corpus:
            for x in i:
                s = s + x[1]
        s = float(s)    
        self.avgdl = s / self.corpus_size
        self.corpus = corpus
        self.df = dictionary.dfs
        self.idf = {}
        self.initialize()

    def initialize(self):
        for word, freq in iteritems(self.df):
            self.idf[word] = math.log(self.corpus_size - freq + 0.5) - math.log(freq + 0.5)

    def get_score(self, query, index, average_idf):
        score = 0
        index_doc = dict(self.corpus[index])
        
        for word,freq in query:
            if word not in index_doc:
                continue
            idf = self.idf[word] if self.idf[word] >= 0 else EPSILON * average_idf
            score += freq*((idf * index_doc[word] * (PARAM_K1 + 1)
                      / (index_doc[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * doc_len[index] / self.avgdl))))
        return score

    def get_scores(self, query, average_idf):
        scores = []
        for index in xrange(self.corpus_size):
            score = self.get_score(query, index, average_idf)
            scores.append(score)
        return scores

